# GrimAge2ADM

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [46]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import numpy as np

## Instantiate model class

In [47]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.GrimAge2ADM)

class GrimAge2ADM(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        return x



In [48]:
model = pya.models.GrimAge2ADM()

## Define clock metadata

In [49]:
model.metadata["clock_name"] = 'grimage2adm'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2022
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Lu, Ake T., et al. \"DNA methylation GrimAge version 2.\" Aging (Albany NY) 14.23 (2022): 9484."
model.metadata["doi"] = "https://doi.org/10.18632/aging.204434"
model.metadata["research_only"] = True
model.metadata["notes"] = None

## Download clock dependencies

In [50]:
logger = pya.logger.Logger()
urls = [
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2_subcomponents.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/datMiniAnnotation3_Gold.csv",
]
dir = "."
for url in urls:
    pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Data found in ./grimage2_subcomponents.csv
|-----------> Data found in ./grimage2.csv
|-----------> Data found in ./datMiniAnnotation3_Gold.csv


## Load features

#### From CSV

In [51]:
df = pd.read_csv('grimage2_subcomponents.csv', index_col=0)
df['Y.pred'].unique()

array(['DNAmGDF_15', 'DNAmB2M', 'DNAmCystatin_C', 'DNAmTIMP_1', 'DNAmadm',
       'DNAmpai_1', 'DNAmleptin', 'DNAmPACKYRS', 'DNAmlog.CRP',
       'DNAmlog.A1C'], dtype=object)

In [52]:
df = df[df['Y.pred'] == 'DNAmadm']
df['feature'] = df['var']
df['coefficient'] = df['beta']
model.features = ['age'] + df['feature'][2:].tolist()

In [53]:
df.head()

,Y.pred,var,beta,feature,coefficient
425,DNAmadm,Intercept,290.169303,Intercept,290.169303
426,DNAmadm,Age,0.943695,Age,0.943695
427,DNAmadm,cg13947317,4.995108,cg13947317,4.995108
428,DNAmadm,cg21272576,5.086187,cg21272576,5.086187
429,DNAmadm,cg03522107,28.640903,cg03522107,28.640903


## Load weights into base model

In [54]:
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'].iloc[0]])

#### Linear model

In [55]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [56]:
reference_df = pd.read_csv('datMiniAnnotation3_Gold.csv', index_col=0)
model.reference_values = [65] + reference_df.loc[model.features[1:]]['gold'].tolist()

## Load preprocess and postprocess objects

In [57]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [58]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [59]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Lu, Ake T., et al. "DNA methylation GrimAge version 2." Aging '
             '(Albany NY) 14.23 (2022): 9484.',
 'clock_name': 'grimage2adm',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.18632/aging.204434',
 'notes': None,
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2022}
reference_values: [65, 0.945983138574711, 0.928628508168924, 0.250997191694227, 0.812900664876887, 0.935420103881463, 0.921443813566399, 0.924647248921382, 0.401166106204001, 0.0694563726960098, 0.583273419793119, 0.90429743695085, 0.0264846427378418, 0.392891324644115, 0.759442858063828, 0.0251869924616306, 0.023407474895155, 0.0358515371617582, 0.0258010644664732, 0.934247150347331, 0.0512855773762465, 0.942843729694158, 0.0203892784279929, 0.922546965145162, 0.646420632424828, 0.910308697016155

## Basic test

In [60]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float).double()
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[  232.9124],
        [   38.8744],
        [   90.0471],
        [ 1517.5669],
        [ -771.0330],
        [-1131.4937],
        [ 1519.1527],
        [ -151.6852],
        [ -417.8726],
        [ -631.4455]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [61]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [62]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: grimage2_subcomponents.csv
Deleted file: datMiniAnnotation3_Gold.csv
Deleted file: grimage2.csv
